In [1]:
import pandas as pd
import numpy as np
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import joblib



In [2]:
# ---------------------
# 1. Load and Inspect Data
# ---------------------
df = pd.read_csv('PB_All_2000_2021.csv', sep=';', decimal=',')
df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y')
df = df.sort_values(by=['id', 'date']).copy()



In [4]:
# ---------------------
# 2. Feature Engineering
# ---------------------
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

pollutants = ['O2', 'NO3', 'NO2', 'SO4', 'PO4', 'CL']

# Convert pollutants to numeric and fill missing values with column mean
for col in pollutants:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    df[col] = df[col].fillna(df[col].mean())



In [5]:
# ---------------------
# 3. Model Preparation
# ---------------------
X = df[['id', 'year']]
y = df[pollutants]

# One-hot encode 'id' column
X_encoded = pd.get_dummies(X, columns=['id'], drop_first=True)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42
)



In [6]:
# ---------------------
# 4. Model Training
# ---------------------
model = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42))
model.fit(X_train, y_train)



MultiOutputRegressor(estimator=RandomForestRegressor(random_state=42))

In [7]:
# ---------------------
# 5. Evaluation
# ---------------------
y_pred = model.predict(X_test)

print("Model Performance on the Test Data:")
for i, pollutant in enumerate(pollutants):
    print(f'{pollutant}:')
    print(f'   MSE: {mean_squared_error(y_test.iloc[:, i], y_pred[:, i]):.2f}')
    print(f'   R2:  {r2_score(y_test.iloc[:, i], y_pred[:, i]):.2f}')
    print()



Model Performance on the Test Data:
O2:
   MSE: 22.86
   R2:  0.01

NO3:
   MSE: 17.77
   R2:  0.52

NO2:
   MSE: 5.69
   R2:  -7.25

SO4:
   MSE: 2688.44
   R2:  0.32

PO4:
   MSE: 0.45
   R2:  0.35

CL:
   MSE: 34492.88
   R2:  0.73



In [8]:
# ---------------------
# 6. Prediction Function
# ---------------------
def predict_pollutants(model, X_encoded_cols, station_id, year_input):
    input_data = pd.DataFrame({'year': [year_input], 'id': [station_id]})
    input_encoded = pd.get_dummies(input_data, columns=['id'])

    # Add any missing columns
    for col in set(X_encoded_cols) - set(input_encoded.columns):
        input_encoded[col] = 0

    input_encoded = input_encoded[X_encoded_cols]  # align order
    prediction = model.predict(input_encoded)[0]
    
    print(f"\nPredicted pollutant levels for station '{station_id}' in {year_input}:")
    for p, val in zip(pollutants, prediction):
        print(f"  {p}: {val:.2f}")



In [9]:
# ---------------------
# 7. Example Prediction
# ---------------------
predict_pollutants(model, X_encoded.columns.tolist(), station_id='22', year_input=2024)




Predicted pollutant levels for station '22' in 2024:
  O2: 14.18
  NO3: 5.01
  NO2: 0.04
  SO4: 128.49
  PO4: 0.49
  CL: 64.78


In [10]:
# ---------------------
# 8. Save the Model
# ---------------------
joblib.dump(model, 'pollution_model.pkl')
joblib.dump(X_encoded.columns.tolist(), "model_columns.pkl")
print('Model and column structure saved successfully!')


Model and column structure saved successfully!
